In [106]:
import keras
import tensorflow as tf
import os,datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle

In [107]:
df = pd.read_csv("./r4.2/http.csv",nrows=100000)
df['date']=pd.to_datetime(df['date'])
df

,id,date,user,pc,url,content
0,{V1Y4-S2IR20QU-6154HFXJ},2010-01-02 06:55:16,LRR0148,PC-4275,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...
1,{Q5R1-T3EF87UE-2395RWZS},2010-01-02 07:00:13,NGF0157,PC-6056,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...
2,{X9O1-O0XW52VO-5806RPHG},2010-01-02 07:03:46,NGF0157,PC-6056,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...
3,{G5S8-U5OG04TE-5299CCTU},2010-01-02 07:05:26,IRM0931,PC-7188,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...
4,{L0R4-A9DH29VP-4553AUWM},2010-01-02 07:05:52,IRM0931,PC-7188,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...
...,...,...,...,...,...,...
99995,{F2B4-J5HG21CP-2472KNWD},2010-01-05 08:24:11,JIM0095,PC-9328,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...
99996,{P4C0-H4AC44QZ-5898ATQE},2010-01-05 08:24:12,BBS0039,PC-9436,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...
99997,{U9Y2-H0WW90NF-1184ROHW},2010-01-05 08:24:12,KAL0395,PC-0004,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...
99998,{Y2Y4-T1KW57MX-2017DWRE},2010-01-05 08:24:12,LDB0090,PC-6824,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...


In [109]:
# Define the function to categorize time frames
def categorize_time_frame(hour):
    if 0 <= hour < 6:
        return 0
    elif 6 <= hour < 12:
        return 1
    elif 12 <= hour < 18:
        return 2
    else:
        return 3

# Apply the function to create the 'time_frame' column
new_df = df.copy()
new_df['time_frame'] = df['date'].dt.hour.apply(categorize_time_frame)
new_df['day'] = df['date'].dt.day
new_df['month'] = df['date'].dt.month
new_df['year'] = df['date'].dt.year

new_df=new_df.drop(columns="date")

le_user = LabelEncoder()
le_user.fit(new_df['user'])
new_df['user'] = le_user.transform(new_df['user'])


#Save the user label encoder
pkl_user_output = open("user_encoder.pkl",'wb')
pickle.dump(le_user, pkl_user_output)


le_pc = LabelEncoder()
le_pc.fit(new_df['pc'])
new_df['pc'] = le_pc.transform(new_df['pc'])

#Save the PC label encoder
pkl_PC_output = open("pc_encoder.pkl",'wb')
pickle.dump(le_pc, pkl_PC_output)

In [102]:
new_df

,id,user,pc,url,content,time_frame,day,month,year
0,{V1Y4-S2IR20QU-6154HFXJ},620,423,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...,1,2,1,2010
1,{Q5R1-T3EF87UE-2395RWZS},707,586,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...,1,2,1,2010
2,{X9O1-O0XW52VO-5806RPHG},707,586,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...,1,2,1,2010
3,{G5S8-U5OG04TE-5299CCTU},467,709,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...,1,2,1,2010
4,{L0R4-A9DH29VP-4553AUWM},467,709,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...,1,2,1,2010
...,...,...,...,...,...,...,...,...,...
99995,{F2B4-J5HG21CP-2472KNWD},497,931,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...,1,5,1,2010
99996,{P4C0-H4AC44QZ-5898ATQE},96,942,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...,1,5,1,2010
99997,{U9Y2-H0WW90NF-1184ROHW},534,0,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...,1,5,1,2010
99998,{Y2Y4-T1KW57MX-2017DWRE},595,681,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...,1,5,1,2010


In [105]:
decoded_users = le_user.inverse_transform(new_df['user'])
decoded_users

array(['LRR0148', 'NGF0157', 'NGF0157', ..., 'KAL0395', 'LDB0090',
       'WTF0387'], dtype=object)